# Content and requrements of this notebook

train models.ipynb is a notebook containing all the code required to train the models
and store them in fitted models

TODO:

-The structured perceptron explained during the course. You can think about adding
or modifiying feature templates to improve results.

-At least one deep learning model.

-Extra exercise
This extra exercise can increment your mark on this deliverable giving you up to 1 extra
point.
Improve the provided code for the structured perceptron. In particular:
• Find where it spends more time during training and inference.
• Improve the code to make it faster.
• Benchmark the improvement.
• Explain your improvements in the pdf file.
Hint: We have seen how cython can be used to dramatically improve the speed of python.
You can benefit from cython to do this

In [26]:
import pandas as pd

train_df = pd.read_csv('data/train_data_ner.csv')
test_df = pd.read_csv('data/test_data_ner.csv')
print(train_df.shape)
print(test_df.shape)

print(f"Number of sentences for train: {train_df['sentence_id'].nunique()}")
print(f"Number of sentences for test: {test_df['sentence_id'].nunique()}")
print("so overall the sentences for test are less but longer...")


print(train_df.head(24))
# print("\nPrint second test sentance:")
# print(test_df[test_df['sentence_id']==1])

#print("\nPrint the 4th sentence for example:")
#print(train_df[train_df['sentence_id']==4])

print(f"Unique tags:\n{train_df['tags'].value_counts()}")



(839149, 3)
(837339, 3)
Number of sentences for train: 38366
Number of sentences for test: 38367
so overall the sentences for test are less but longer...
    sentence_id          words   tags
0             0      Thousands      O
1             0             of      O
2             0  demonstrators      O
3             0           have      O
4             0        marched      O
5             0        through      O
6             0         London  B-geo
7             0             to      O
8             0        protest      O
9             0            the      O
10            0            war      O
11            0             in      O
12            0           Iraq  B-geo
13            0            and      O
14            0         demand      O
15            0            the      O
16            0     withdrawal      O
17            0             of      O
18            0        British  B-gpe
19            0         troops      O
20            0           from      O
21        

# Part Alana Zoloeva

# Part Ralitsa Dimitrova

In [17]:
import utils
import skseq
import skseq.sequences.structured_perceptron as spc

In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
from utils.structured_perceptrone_utils import NewPostagCorpus

In [20]:
corpus = NewPostagCorpus()
train_seq = corpus.read_sequence_list_csv("data/train_data_ner.csv")

# USe a new corpus so the train data one is not poluted, but the same IDs are used for the existing words in test and train
corpus1 = NewPostagCorpus()
corpus1.word_dict = corpus.word_dict
corpus1.tag_dict = corpus.tag_dict
test_seq = corpus1.read_sequence_list_csv("data/test_data_ner.csv", update_dict=False)

word_dict = train_seq.x_dict
tag_dict  = train_seq.y_dict

In [309]:
len(train_seq)

38364

In [310]:
train_seq.y_dict

{'O': 0,
 'B-geo': 1,
 'B-gpe': 2,
 'B-tim': 3,
 'B-org': 4,
 'I-geo': 5,
 'B-per': 6,
 'I-per': 7,
 'I-org': 8,
 'B-art': 9,
 'I-art': 10,
 'I-tim': 11,
 'I-gpe': 12,
 'B-nat': 13,
 'I-nat': 14,
 'B-eve': 15,
 'I-eve': 16}

In [311]:
# vocabulary size
len(train_seq.x_dict)

31965

In [312]:
sequence = train_seq[1]
sequence.to_words(sequence_list=train_seq)

'Helicopter/O gunships/O Saturday/B-tim pounded/O militant/O hideouts/O in/O the/O Orakzai/B-geo tribal/O region/O ,/O where/O many/O Taliban/B-org militants/O are/O believed/O to/O have/O fled/O to/O avoid/O an/O earlier/O military/O offensive/O in/O nearby/O South/B-geo Waziristan/I-geo ./O '

## 0. Build a NER perceptrone similar to the POS one, which works with loading .csv files (For baseline)

In [313]:
feature_mapper = skseq.sequences.id_feature.IDFeatures(train_seq)

In [315]:
feature_mapper.build_features()

In [316]:
len(feature_mapper.feature_dict)

39783

In [317]:
len(feature_mapper.feature_list)

38364

In [319]:
m = 1
len(feature_mapper.feature_list[m])

4

In [320]:
id_seq = 1
feature_mapper.dataset[id_seq]

22/0 23/0 24/3 25/0 26/0 27/0 11/0 9/0 28/1 29/0 30/0 31/0 32/0 33/0 34/4 35/0 36/0 37/0 7/0 3/0 38/0 7/0 39/0 40/0 41/0 42/0 43/0 11/0 44/0 45/1 46/5 21/0 

In [321]:
len(feature_mapper.dataset[id_seq])

32

In [322]:
print ("Initial features:",     feature_mapper.feature_list[id_seq][0])
print ("Transition features:",  feature_mapper.feature_list[id_seq][1])
print ("Final features:",       feature_mapper.feature_list[id_seq][2])
print ("Emission features:",    feature_mapper.feature_list[id_seq][3])

Initial features: [[0]]
Transition features: [[3], [32], [34], [3], [3], [3], [3], [9], [11], [3], [3], [3], [3], [44], [46], [3], [3], [3], [3], [3], [3], [3], [3], [3], [3], [3], [3], [3], [9], [58], [59]]
Final features: [[28]]
Emission features: [[29], [30], [31], [33], [35], [36], [15], [13], [37], [38], [39], [40], [41], [42], [43], [45], [47], [48], [10], [5], [49], [10], [50], [51], [52], [53], [54], [15], [55], [56], [57], [27]]


In [327]:
feature_type = ["Initial features", "Transition features", "Final features", "Emission features"]
inv_feature_dict = {word: pos for pos, word in feature_mapper.feature_dict.items()}

In [325]:
id_seq = 6
seq = train_seq[id_seq]

In [361]:
from utils.structured_perceptrone_utils import show_feats

In [326]:
show_feats(feature_mapper, seq, inv_feature_dict, feature_type)

Initial features
	 [0]
		 init_tag:O


Transition features
	 [3]
		 prev_tag:O::O
	 [3]
		 prev_tag:O::O
	 [3]
		 prev_tag:O::O
	 [3]
		 prev_tag:O::O
	 [3]
		 prev_tag:O::O
	 [3]
		 prev_tag:O::O


Final features
	 [28]
		 final_prev_tag:O


Emission features
	 [132]
		 id:Some::O
	 [133]
		 id:1,27,000::O
	 [106]
		 id:people::O
	 [47]
		 id:are::O
	 [134]
		 id:known::O
	 [135]
		 id:dead::O
	 [27]
		 id:.::O




In [328]:
sp = spc.StructuredPerceptron(corpus.word_dict, corpus.tag_dict, feature_mapper)
sp.num_epochs = 5

In [329]:
sp.state_labels

{'O': 0,
 'B-geo': 1,
 'B-gpe': 2,
 'B-tim': 3,
 'B-org': 4,
 'I-geo': 5,
 'B-per': 6,
 'I-per': 7,
 'I-org': 8,
 'B-art': 9,
 'I-art': 10,
 'I-tim': 11,
 'I-gpe': 12,
 'B-nat': 13,
 'I-nat': 14,
 'B-eve': 15,
 'I-eve': 16}

In [331]:
len(sp.parameters)

39783

In [333]:
sp.fit(feature_mapper.dataset, 5)

Epoch: 0 Accuracy: 0.893909
Epoch: 1 Accuracy: 0.931787
Epoch: 2 Accuracy: 0.941169
Epoch: 3 Accuracy: 0.946112
Epoch: 4 Accuracy: 0.949991


In [335]:
sp.parameters.sum()

100124.2

In [336]:
sp.save_model("fitted_models/perceptron_baseline_")

In [337]:
pred_train = sp.viterbi_decode_corpus(train_seq)

In [348]:
# IMPORTANT! Using the proposed way in lectures, so that there are no unnown words, althouh this is unlikely in 
# a real scenario: (For the imporvements of the perceptrone for NER I have put handling for unknown words)
test_seq_nn = corpus.read_sequence_list_csv("data/test_data_ner.csv", update_dict=True)

pred_test = sp.viterbi_decode_corpus(test_seq_nn)

In [349]:
from utils.structured_perceptrone_utils import evaluate_corpus

In [350]:
eval_train = evaluate_corpus(train_seq.seq_list, pred_train)
eval_test = evaluate_corpus(test_seq_nn.seq_list, pred_test)
print("SP -  Accuracy Train: %.3f Test: %.3f"%(eval_train, eval_test))

SP -  Accuracy Train: 0.962 Test: 0.876


## 1. Train a structured perceptrone with custom features

In [362]:
from utils.structured_perceptrone_utils import NERFeatures

In [288]:
feature_mapper_ner = NERFeatures(train_seq)
feature_mapper_ner.build_features()

In [289]:
# new strucutured perceptrone:
sp_1 = spc.StructuredPerceptron(word_dict, tag_dict, feature_mapper_ner)
sp_1.num_epochs = 5

In [291]:
len(feature_mapper_ner.feature_dict) # more than the baseline!

139360

In [293]:
# train the new perceptrone:
sp_1.fit(feature_mapper_ner.dataset, 5)

Epoch: 0 Accuracy: 0.944704
Epoch: 1 Accuracy: 0.954096
Epoch: 2 Accuracy: 0.957750
Epoch: 3 Accuracy: 0.959741
Epoch: 4 Accuracy: 0.960863


In [294]:
sp_1.save_model("fitted_models/perceptron_1_")

In [295]:
sp_1.parameters.sum()

103343.40000000001

In [296]:
pred_train_1 = sp_1.viterbi_decode_corpus(train_seq)

In [297]:
index_to_feature = {v: k for k, v in feature_mapper_ner.feature_dict.items()}
print(len(index_to_feature))
for seq in test_seq.seq_list:
    for idx in seq.x:
        if idx not in index_to_feature:
            print("Unknown feature index:", idx)

139360


In [298]:
for i, seq in enumerate(test_seq.seq_list):
    if len(seq.x) == 0:
        print(f"Empty feature sequence at index {i}")

Empty feature sequence at index 13539
Empty feature sequence at index 29617
Empty feature sequence at index 29859


In [300]:
print(test_seq.seq_list[13539])

In [304]:
fixed_test_seq_list = []

for seq in test_seq.seq_list:
    if len(seq.x) == 0:
        # Replace empty input with one UNK word and dummy label
        dummy_x = ["<UNK>"]
        dummy_y = [0]  # Arbitrary, won't be used in viterbi
        new_seq = Sequence(x=dummy_x, y=dummy_y)
        fixed_test_seq_list.append(new_seq)
    else:
        fixed_test_seq_list.append(seq)

from skseq.sequences.sequence_list import SequenceList  

fixed_test_dataset = SequenceList(x_dict=test_seq.x_dict, y_dict=test_seq.y_dict)
fixed_test_dataset.seq_list = fixed_test_seq_list

In [305]:
pred_test_1 = sp_1.viterbi_decode_corpus(fixed_test_dataset)

In [307]:
eval_train_1 = evaluate_corpus(train_seq.seq_list, pred_train_1)
eval_test_1 = evaluate_corpus(fixed_test_dataset.seq_list, pred_test_1)
print("SP -  Accuracy Train: %.3f Test: %.3f"%(eval_train_1, eval_test_1))

SP -  Accuracy Train: 0.962 Test: 0.978


## 2. Second test with NER handcrafter fetures:

In [363]:
from utils.structured_perceptrone_utils import NERFeatures_2

In [354]:
feature_mapper_ner_2 = NERFeatures_2(train_seq)
feature_mapper_ner_2.build_features()

In [355]:
len(feature_mapper_ner_2.feature_dict) #looots

223395

In [356]:
sp_2 = spc.StructuredPerceptron(word_dict, tag_dict, feature_mapper_ner_2)
sp_2.num_epochs = 5

In [357]:
len(sp_2.parameters)

223395

In [358]:
sp_2.fit(feature_mapper_ner_2.dataset, 5)

Epoch: 0 Accuracy: 0.947189
Epoch: 1 Accuracy: 0.955523
Epoch: 2 Accuracy: 0.958542
Epoch: 3 Accuracy: 0.959905
Epoch: 4 Accuracy: 0.961356


In [359]:
sp_2.save_model("fitted_models/perceptron_2_")

## 3. Test 3 for handcrafted features for NER

In [376]:
from utils.structured_perceptrone_utils import NERFeatures_3

In [369]:
feature_mapper_ner_3 = NERFeatures_3(train_seq)
feature_mapper_ner_3.build_features()

In [370]:
len(feature_mapper_ner_3.feature_dict) # less than previous approach

114412

In [371]:
sp_3 = spc.StructuredPerceptron(word_dict, tag_dict, feature_mapper_ner_3)
sp_3.num_epochs = 5

In [373]:
sp_3.fit(feature_mapper_ner_3.dataset, 5)

Epoch: 0 Accuracy: 0.943159
Epoch: 1 Accuracy: 0.953967
Epoch: 2 Accuracy: 0.957306
Epoch: 3 Accuracy: 0.959520
Epoch: 4 Accuracy: 0.961097


In [374]:
sp_3.save_model("fitted_models/perceptron_3_")

## 4. Test 4 for handcrafted features for NER

In [25]:
from utils.structured_perceptrone_utils import NERFeatures_4

In [26]:
feature_mapper_ner_4 = NERFeatures_4(train_seq)
feature_mapper_ner_4.build_features()

In [27]:
len(feature_mapper_ner_4.feature_dict) # less than previous approach

139416

In [29]:
sp_4 = spc.StructuredPerceptron(word_dict, tag_dict, feature_mapper_ner_4)
sp_4.num_epochs = 5

In [30]:
sp_4.fit(feature_mapper_ner_4.dataset, 5)

Epoch: 0 Accuracy: 0.944617
Epoch: 1 Accuracy: 0.954481
Epoch: 2 Accuracy: 0.957610
Epoch: 3 Accuracy: 0.959663
Epoch: 4 Accuracy: 0.961082


In [31]:
sp_4.save_model("fitted_models/perceptron_4_")

## 5. Test 5 for handcrafted features for NER

In [32]:
from utils.structured_perceptrone_utils import NERFeatures_5

In [33]:
feature_mapper_ner_5 = NERFeatures_5(train_seq)
feature_mapper_ner_5.build_features()

In [34]:
len(feature_mapper_ner_5.feature_dict) # less than previous approach

139388

In [35]:
sp_5 = spc.StructuredPerceptron(word_dict, tag_dict, feature_mapper_ner_5)
sp_5.num_epochs = 5

In [36]:
sp_5.fit(feature_mapper_ner_5.dataset, 5)

Epoch: 0 Accuracy: 0.944945
Epoch: 1 Accuracy: 0.954311
Epoch: 2 Accuracy: 0.957469
Epoch: 3 Accuracy: 0.959819
Epoch: 4 Accuracy: 0.961035


In [37]:
sp_5.save_model("fitted_models/perceptron_5_")

## 6. Test 6 for handcrafted features for NER

In [43]:
from utils.structured_perceptrone_utils import NERFeatures_6

In [44]:
feature_mapper_ner_6 = NERFeatures_6(train_seq)
feature_mapper_ner_6.build_features()

In [45]:
len(feature_mapper_ner_6.feature_dict) # less than previous approach

139419

In [46]:
sp_6 = spc.StructuredPerceptron(word_dict, tag_dict, feature_mapper_ner_6)
sp_6.num_epochs = 5

In [47]:
sp_6.fit(feature_mapper_ner_6.dataset, 5)

Epoch: 0 Accuracy: 0.944688
Epoch: 1 Accuracy: 0.954412
Epoch: 2 Accuracy: 0.957566
Epoch: 3 Accuracy: 0.959478
Epoch: 4 Accuracy: 0.961124


In [48]:
sp_6.save_model("fitted_models/perceptron_6_")